# Stock reccomender
Steve Smith
Jan 2025

The purpose of this is to build an AI agent that will reccomend stocks, 

  item  language  number_issues  recommendation_score
0    A       0.2            0.7              1.000000
1    B       0.8            0.1              0.391862
2    C       0.3            0.5              0.965843


In [ ]:
# get user langauges
import requests

def get_user_languages(username, token=None):
    """Fetch unique programming languages from a GitHub user's repositories."""
    headers = {"Authorization": f"token {token}"} if token else {}

    # Get user repositories
    repos_url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(repos_url, headers=headers)
    repos = response.json()

    languages = set()

    # Fetch languages for each repo
    for repo in repos:
        lang_url = repo["languages_url"]
        lang_response = requests.get(lang_url, headers=headers)
        repo_languages = lang_response.json().keys()
        languages.update(repo_languages)

    return languages

# Replace 'octocat' with any GitHub username
username = "octocat"  
languages = get_user_languages(username)

print(f"{username} uses: {languages}")


In [35]:
from configgithub import API_KEY


In [33]:
import numpy as np

def summarize_user_langs(tracking_dict):
    user_langs_df = pd.Series(tracking_dict).reset_index()
    user_langs_df.columns = ['Language', 'Bites']
    total_bytes=np.sum(user_langs_df['Bites'])

    user_langs_df['prop_bites']=user_langs_df['Bites']/total_bytes
    return user_langs_df



In [39]:
from github import Github

# Authenticate (Optional: Use a GitHub token for higher rate limits)
g = Github(API_KEY)

# Languages are stored on a per-repo basis as language : bites
# Need to loop through each USER's repo and tally up total bites per language
def get_languages(username):
    user = g.get_user(username)
    tracking_dict={}

    # May want to break this into def?
    # also move any per-repo GETS here
    for i in user.get_repos():
        langs= i.get_languages()
        for key, value in langs.items():
            tracking_dict[key] = tracking_dict.get(key, 0) + value
    
    # convert to df and tally prop of bites per langauge
    user_l_df= summarize_user_langs(tracking_dict)
    
    return user_l_df

languages_sbs87 = get_languages("sbs87")
languages_octo = get_languages("octocat")



In [58]:
languages_octo['user']='octo'
languages_sbs87['user']='sbs87'

test_in=pd.concat([languages_octo,languages_sbs87])[["Language","user","prop_bites"]].pivot(index='user', columns='Language', values='prop_bites').fillna(0).reset_index()

In [59]:
similarity_matrix = cosine_similarity(test_in.iloc[:, 1:])
test_in["recommendation_score"] = similarity_matrix[0]
test_in

Language,user,C++,CSS,HTML,JavaScript,Jupyter Notebook,Perl,PostScript,Python,R,Ruby,Shell,TeX,recommendation_score
0,octo,0.000000,0.066412,0.019270,0.000213,0.000000,0.00000,0.000000,0.000000,0.000000,0.910062,0.004042,0.000000,1.00000
1,sbs87,0.000166,0.000107,0.639653,0.000149,0.329618,0.00144,0.010033,0.006111,0.004195,0.000000,0.002339,0.006188,0.01879


In [44]:
# github reccomender DO NOT COMMIT
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Sample dataset
df = pd.DataFrame({
    "user": ["A", "B", "C"],
    "language": [0.2, 0.8, 0.3],
    "number_issues": [0.7, 0.1, 0.5]
})

# Compute similarity
similarity_matrix = cosine_similarity(df.iloc[:, 1:])
df["recommendation_score"] = similarity_matrix[0]  # Similarity to item A

print(df)


  user  language  recommendation_score
0    A       0.2                   1.0
1    B       0.8                   1.0
2    C       0.3                   1.0


In [ ]:
def get_users_and_repos(since=0, per_page=5, token=None):
    """Fetch a list of users and their public repositories."""
    headers = {"Authorization": f"token {token}"} if token else {}
    
    # Step 1: Get public users
    users_url = f"https://api.github.com/users?since={since}&per_page={per_page}"
    users_response = requests.get(users_url, headers=headers)
    users = users_response.json()

    user_repos = {}

    # Step 2: Get each user's repositories
    for user in users:
        username = user["login"]
        repos_url = f"https://api.github.com/users/{username}/repos"
        repos_response = requests.get(repos_url, headers=headers)
        repos = repos_response.json()

        # Extract repo names
        user_repos[username] = [repo["name"] for repo in repos]

    return user_repos

# Fetch 5 users and their repos
users_repos = get_users_and_repos(per_page=5)
print(users_repos)


In [ ]:
# deep learning
import tensorflow_recommenders as tfrs
import tensorflow as tf

# Load MovieLens dataset
ratings = tf.data.Dataset.from_tensor_slices({
    "user_id": ["1", "2", "3"],
    "movie_id": ["101", "102", "103"],
    "rating": [4.0, 5.0, 3.0]
})

# Define user and movie embeddings
user_model = tf.keras.Sequential([tf.keras.layers.StringLookup(vocabulary=["1", "2", "3"]), tf.keras.layers.Embedding(3, 16)])
movie_model = tf.keras.Sequential([tf.keras.layers.StringLookup(vocabulary=["101", "102", "103"]), tf.keras.layers.Embedding(3, 16)])

# Train a ranking model
model = tfrs.models.Model(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))


# Fetch data

In [8]:
from tensortrade.data.cdd import CryptoDataDownload

import numpy as np
import pandas as pd
pd.options.mode.use_inf_as_na = True

def prepare_data(df):
    df['volume'] = np.int64(df['volume'])
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')
    return df

#   CryptoDataDownload  Parameters
#         ----------
#         exchange_name : str
#             The name of the exchange.
#         base_symbol : str
#             The base symbol fo the cryptocurrency pair.
#         quote_symbol : str
#             The quote symbol fo the cryptocurrency pair.
#         timeframe : {"d", "h", "m"}
#             The timeframe to collect data from.
#         include_all_volumes : bool, optional
#             Whether or not to include both base and quote volume.

#         Returns
#         -------
#         `pd.DataFrame`
#             A open, high, low, close and volume for the specified exchange and
#             cryptocurrency pair.
#         """

def fetch_data(exchange_name="Bitfinex",quote_symbol="BTC",timeframe="1h"):
    # simulated data for now?
    cdd = CryptoDataDownload()
    bitfinex_data = cdd.fetch(exchange_name, "USD", quote_symbol, timeframe)
    bitfinex_data = bitfinex_data[['date', 'open', 'high', 'low', 'close', 'volume']]
    bitfinex_data = prepare_data(bitfinex_data)
    return bitfinex_data

def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp from "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    return prepare_data(df)

#TODO : play with data size parameters such as timefrome (increases number of rows)
#TODO: make sure parameters confrom to the acceptable set
market_data = fetch_data(timeframe="1h")
market_data

/var/folders/vv/lg2w1mmx79b93kh1fxk_v5p00000gn/T/ipykernel_13003/3395647669.py:5: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


,date,open,high,low,close,volume
0,2018-05-15 06:00 AM,8723.8,8793.0,8714.9,8739.0,8988053
1,2018-05-15 07:00 AM,8739.0,8754.8,8719.3,8743.0,2288904
2,2018-05-15 08:00 AM,8743.0,8743.1,8653.2,8723.7,8891773
3,2018-05-15 09:00 AM,8723.7,8737.8,8701.2,8708.1,2054868
4,2018-05-15 10:00 AM,8708.1,8855.7,8695.8,8784.4,17309722
...,...,...,...,...,...,...
58178,2025-01-02 09:00 PM,97378.0,97394.0,97128.0,97262.0,174426
58179,2025-01-02 10:00 PM,97261.0,97261.0,96777.0,97000.0,1611671
58180,2025-01-02 11:00 PM,96984.0,97106.0,96871.0,97035.0,490811
58181,2025-01-03 12:00 AM,97035.0,97101.0,96872.0,96911.0,2174059
